In [14]:
from telemetry.mavlogparse import Telemetry
import json
import ipywidgets as widgets
from IPython.display import display

## FlightTime Text File Processing

In [15]:
with open('flightTimes.txt') as f:
    lines = f.readlines()
lines

['Processing log 00000063.BIN\n',
 'In air at Mon Jun 13 10:01:58 2022 (percent 15% groundspeed 3.1)\n',
 'On ground at Mon Jun 13 10:03:25 2022 (percent 22.3% groundspeed 2.8  time=87.0 seconds)\n',
 'In air at Mon Jun 13 10:03:30 2022 (percent 23% groundspeed 3.0)\n',
 'On ground at Mon Jun 13 10:04:54 2022 (percent 29.4% groundspeed 2.9  time=84.0 seconds)\n',
 'In air at Mon Jun 13 10:05:00 2022 (percent 30% groundspeed 3.2)\n',
 'On ground at Mon Jun 13 10:05:56 2022 (percent 34.3% groundspeed 2.5  time=56.0 seconds)\n',
 'In air at Mon Jun 13 10:06:19 2022 (percent 36% groundspeed 3.0)\n',
 'On ground at Mon Jun 13 10:06:46 2022 (percent 38.3% groundspeed 2.5  time=27.0 seconds)\n',
 'In air at Mon Jun 13 10:07:06 2022 (percent 40% groundspeed 3.1)\n',
 'On ground at Mon Jun 13 10:07:37 2022 (percent 42.4% groundspeed 2.8  time=31.0 seconds)\n',
 'In air at Mon Jun 13 10:07:46 2022 (percent 43% groundspeed 3.1)\n',
 'On ground at Mon Jun 13 10:08:33 2022 (percent 46.8% groundspee

In [56]:
ft = 'Flight time'
tta = 'Total time in air'
tdt = 'Total distance travelled'
iaa = 'In air at'
dateTest = False
allRows, row = [], [] 
for l in lines:
    
    if iaa in l and dateTest == False:
        row, dateTest = [], True
        rp = (l).rstrip("\n")
        dayWeek, dayText, time24, year = rp[10:13], rp[14:20], rp[21:29], rp[30:34]
        row.extend([dayWeek, dayText, time24, year])
    if tta in l:
        rp = (l).rstrip("\n")        
        totTime = rp[19:]
        row.append(totTime)
    if tdt in l: 
        dateTest = False
        rp = (l).rstrip("\n")
        print(rp)
        totDist = rp[26:32]
        row.append(totDist)
        allRows.append(row)

allRows


Total distance travelled: 6409.1 meters
Total distance travelled: 4602.8 meters
Total distance travelled: 4595.4 meters
Total distance travelled: 1242.0 meters
Total distance travelled: 1372.6 meters
Total distance travelled: 1385.7 meters
Total distance travelled: 1364.9 meters
Total distance travelled: 1367.8 meters
Total distance travelled: 1381.4 meters
Total distance travelled: 1364.4 meters
Total distance travelled: 1376.6 meters
Total distance travelled: 1348.3 meters
Total distance travelled: 2702.7 meters
Total distance travelled: 1353.4 meters
Total distance travelled: 1354.6 meters
Total distance travelled: 1346.0 meters
Total distance travelled: 1351.4 meters
Total distance travelled: 1362.5 meters
Total distance travelled: 1349.1 meters
Total distance travelled: 2696.6 meters
Total distance travelled: 1351.8 meters
Total distance travelled: 1357.1 meters
Total distance travelled: 1345.5 meters
Total distance travelled: 1351.8 meters
Total distance travelled: 1345.7 meters


[['Mon', 'Jun 13', '10:01:58', '2022', '13:55', '6409.1'],
 ['Mon', 'Jun 13', '10:54:41', '2022', '8:54', '4602.8'],
 ['Mon', 'Jun 13', '15:37:35', '2022', '8:55', '4595.4'],
 ['Wed', 'Jun 15', '11:10:30', '2022', '3:06', '1242.0'],
 ['Wed', 'Jun 15', '11:40:06', '2022', '2:22', '1372.6'],
 ['Wed', 'Jun 15', '12:13:05', '2022', '2:19', '1385.7'],
 ['Wed', 'Jun 15', '13:48:17', '2022', '2:24', '1364.9'],
 ['Wed', 'Jun 15', '13:59:26', '2022', '2:14', '1367.8'],
 ['Thu', 'Jun 16', '10:36:54', '2022', '2:23', '1381.4'],
 ['Fri', 'Jun 17', '08:58:32', '2022', '2:23', '1364.4'],
 ['Fri', 'Jun 17', '09:14:16', '2022', '2:16', '1376.6'],
 ['Fri', 'Jun 17', '09:28:16', '2022', '1:51', '1348.3'],
 ['Fri', 'Jun 17', '09:36:10', '2022', '3:37', '2702.7'],
 ['Fri', 'Jun 17', '10:02:54', '2022', '1:47', '1353.4'],
 ['Fri', 'Jun 17', '10:21:16', '2022', '1:53', '1354.6'],
 ['Fri', 'Jun 17', '10:30:07', '2022', '1:47', '1346.0'],
 ['Sun', 'Jun 19', '08:47:55', '2022', '1:52', '1351.4'],
 ['Sun', 'Jun

## dronekit testing
### python library
- pip install dronekit
- pip install dronekit-sitl

In [18]:
#@markdown ### Select Fields:
field_selection_method = 'Use default fields' #@param ['Use default fields', 'Extract available fields', 'Upload json fields file']
if field_selection_method == 'Use default fields':
    fields = Telemetry.DEFAULT_FIELDS
elif field_selection_method == 'Extract available fields':
    Telemetry.get_useful_fields(tlogs)
    print('\nModify useful.json in the file explorer from the sidebar to only')
    print('include the fields you want. (Double-click to view/edit)\n')
    button = widgets.Button(description="Done")
    def submit(b):
        global fields
        with open('useful.json') as file:
            fields = json.load(file)
        print('\nFields successfully loaded')
    button.on_click(submit)
    display(button)
else:
    print('Upload json file of desired fields:')
    fields = json.loads(upload_one_file())

In [22]:
fields

{'VFR_HUD': ['heading', 'alt', 'climb'],
 'VIBRATION': ['vibration_x', 'vibration_y', 'vibration_z'],
 'SCALED_IMU2': ['xacc', 'xgyro', 'yacc', 'ygyro', 'zacc', 'zgyro'],
 'ATTITUDE': ['roll', 'rollspeed', 'pitch', 'pitchspeed', 'yaw', 'yawspeed'],
 'SCALED_PRESSURE2': ['temperature']}

In [28]:
tlogs = 'home//pi//dev//uav//data-alignment//2022-06-15 12-12-21.tlog'

In [ ]:
#@markdown ### Combine Output CSVs?
combined_CSV_output_filename = "Don't Combine" #@param ["Don't Combine"] {allow-input: true}
if combined_CSV_output_filename == "Don't Combine":
    combined_CSV_output_filename = None
Telemetry.logs_to_csv(combined_CSV_output_filename, tlogs, fields)
print('\nRight click output csv(s) to download (from sidebar at left)')

In [ ]:
csvPath = 'data//2022-06-17_08-58-07_allParams.csv'
df = Telemetry.csv_to_df('2022-06-15 12-12-21.csv', timezone='US/Eastern')

## Official Workflow
- data//2022-06-17_08-58-07_allParams.csv => 237 useful columns
- data//ALLDATA-2022-06-17 08-58-07.csv => mismatched columns error
- removed 45 non conforming records in ALLDATA
- Now 246 useful columns

In [12]:
import pandas as pd

In [13]:
# load CSV
# Testing various versions
inputPath = 'data//ALLDATA-2022-06-17 08-58-07.csv'
file = open(inputPath)
CSVlines = len(file.readlines())
print(CSVlines)
#df = pd.read_csv(inputPath, skiprows=i, nrows=self.numRec)
df = pd.read_csv(inputPath, low_memory=False)
df.shape

19317


(19316, 496)

In [19]:
# show all columns to drop extras at end
for c in df.columns:
    print(c)

timestamp
ADSB_VEHICLE.ICAO_address
ADSB_VEHICLE.lat
ADSB_VEHICLE.lon
ADSB_VEHICLE.altitude_type
ADSB_VEHICLE.altitude
ADSB_VEHICLE.heading
ADSB_VEHICLE.hor_velocity
ADSB_VEHICLE.ver_velocity
ADSB_VEHICLE.callsign
ADSB_VEHICLE.emitter_type
ADSB_VEHICLE.tslc
ADSB_VEHICLE.flags
ADSB_VEHICLE.squawk
AHRS.omegaIx
AHRS.omegaIy
AHRS.omegaIz
AHRS.error_rp
AHRS.error_yaw
AHRS2.roll
AHRS2.pitch
AHRS2.yaw
AHRS3.roll
AHRS3.pitch
AHRS3.yaw
AHRS3.altitude
AHRS3.lat
AHRS3.lng
ATTITUDE.time_boot_ms
ATTITUDE.roll
ATTITUDE.pitch
ATTITUDE.yaw
ATTITUDE.rollspeed
ATTITUDE.pitchspeed
ATTITUDE.yawspeed
AUTOPILOT_VERSION_REQUEST.target_component
BATTERY2.voltage
BATTERY2.current_battery
BATTERY_STATUS.voltages
BATTERY_STATUS.id
BATTERY_STATUS.current_battery
BATTERY_STATUS.current_consumed
BATTERY_STATUS.energy_consumed
BATTERY_STATUS.battery_remaining
BATTERY_STATUS.temperature
COMMAND_ACK.command
COMMAND_LONG.target_component
COMMAND_LONG.command
COMMAND_LONG.param1
COMMAND_LONG.param3
DISTANCE_SENSOR.time_

In [24]:
# remove 250 extraneous columns at the end
dfDrop = df.drop(df.iloc[:, 246:496],axis = 1)
dfDrop.shape

(19316, 246)

In [25]:
dfDrop.columns

Index(['timestamp', 'ADSB_VEHICLE.ICAO_address', 'ADSB_VEHICLE.lat',
       'ADSB_VEHICLE.lon', 'ADSB_VEHICLE.altitude_type',
       'ADSB_VEHICLE.altitude', 'ADSB_VEHICLE.heading',
       'ADSB_VEHICLE.hor_velocity', 'ADSB_VEHICLE.ver_velocity',
       'ADSB_VEHICLE.callsign',
       ...
       'VIBRATION.clipping_2', 'UNKNOWN_1', 'UNKNOWN_2', 'UNKNOWN_3',
       'UNKNOWN_4', 'UNKNOWN_5', 'UNKNOWN_6', 'UNKNOWN_7', 'UNKNOWN_8',
       'UNKNOWN_9'],
      dtype='object', length=246)

In [30]:
dfDrop.head()

,timestamp,ADSB_VEHICLE.ICAO_address,ADSB_VEHICLE.lat,ADSB_VEHICLE.lon,ADSB_VEHICLE.altitude_type,ADSB_VEHICLE.altitude,ADSB_VEHICLE.heading,ADSB_VEHICLE.hor_velocity,ADSB_VEHICLE.ver_velocity,ADSB_VEHICLE.callsign,...,VIBRATION.clipping_2,UNKNOWN_1,UNKNOWN_2,UNKNOWN_3,UNKNOWN_4,UNKNOWN_5,UNKNOWN_6,UNKNOWN_7,UNKNOWN_8,UNKNOWN_9
0,1655470712,12607118,419586944,-830713344,0,10972800,24371,18777,65,JZA741,...,0.112403,245,0,236.080002,0.057993,1241430613,0.021145,0.033484,0.032791,0
1,1655470712,12607118,419586944,-830713344,0,10972800,24371,18777,65,JZA741,...,0.112403,245,0,236.080002,0.057993,1241430613,0.021145,0.033484,0.032791,0
2,1655470712,12607118,419586944,-830713344,0,10972800,24371,18777,65,JZA741,...,0.112403,245,0,236.080002,0.057993,1241430613,0.021145,0.033484,0.032791,0
3,1655470712,12607118,419586944,-830713344,0,10972800,24371,18777,65,JZA741,...,0.112403,245,0,236.080002,0.057993,1241430613,0.021145,0.033484,0.032791,0
4,1655470712,12607118,419586944,-830713344,0,10972800,24371,18777,65,JZA741,...,0.112403,245,0,236.080002,0.057993,1241430613,0.021145,0.033484,0.032791,0


In [34]:
# create new dataframe with only ['VFR_HUD.groundspeed','VFR_HUD.heading'] 
dfGPS = dfDrop[['VFR_HUD.groundspeed','VFR_HUD.heading']]
dfDropGPS = dfGPS.dropna()
dfDropGPS.head()

,VFR_HUD.groundspeed,VFR_HUD.heading
0,425460676,-831951825
1,425460676,-831951825
2,425460676,-831951825
3,425460676,-831951825
4,425460676,-831951825


In [36]:
# format geographic coordinates
def addDec(coord):
    test = float(coord)
    if test > 0:
        cStr = str(coord)[:-2]
        return float((f'{cStr[:2]}.{cStr[2:]}'))
    if test < 0:
        cStr = str(coord)[:-2]
        return float((f'{cStr[:3]}.{cStr[2:]}'))

def fixCoords(df1, coordFields):
    for c in coordFields:
        df1[c] = df1[c].apply(addDec)
    return df1

# coord = -831947797.0
# #print(addDec(coord))

# coordFields = ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon', 'AHRS3.lat', 'AHRS3.lng']
# # Coord fields not contain valid geometry 
# # 'GLOBAL_POSITION_INT.lat', 'GLOBAL_POSITION_INT.lon', 'TERRAIN_REPORT.lat','TERRAIN_REPORT.lon' ]

# dfFormat = fixCoords(df, coordFields)
# dfFormat.head()

In [37]:
# 'VFR_HUD.groundspeed','VFR_HUD.heading'
dfDropGPS['VFR_HUD.heading'] = dfDropGPS['VFR_HUD.heading'].apply(addDec)
dfDropGPS['VFR_HUD.groundspeed'] = dfDropGPS['VFR_HUD.groundspeed'].apply(addDec)
dfDropGPS.head()

,VFR_HUD.groundspeed,VFR_HUD.heading
0,42.54606,-83.319518
1,42.54606,-83.319518
2,42.54606,-83.319518
3,42.54606,-83.319518
4,42.54606,-83.319518


In [39]:
dfDropGPS.to_csv('data//GPS-VFR_HUD.csv')

In [41]:
# 'ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon'
# create new dataframe with only ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon'] 
dfGPS = dfDrop[['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon']]
dfDropGPS = dfGPS.dropna()
dfDropGPS.head()

,ADSB_VEHICLE.lat,ADSB_VEHICLE.lon
0,419586944,-830713344
1,419586944,-830713344
2,419586944,-830713344
3,419586944,-830713344
4,419586944,-830713344


In [42]:
# 'VFR_HUD.groundspeed','VFR_HUD.heading'
dfDropGPS['ADSB_VEHICLE.lat'] = dfDropGPS['ADSB_VEHICLE.lat'].apply(addDec)
dfDropGPS['ADSB_VEHICLE.lon'] = dfDropGPS['ADSB_VEHICLE.lon'].apply(addDec)
dfDropGPS.head()

,ADSB_VEHICLE.lat,ADSB_VEHICLE.lon
0,41.95869,-83.307133
1,41.95869,-83.307133
2,41.95869,-83.307133
3,41.95869,-83.307133
4,41.95869,-83.307133


In [43]:
dfDropGPS.to_csv('data//GPS-ADSB_VEHICLE.csv')

In [ ]:
coordFields = ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon', 'AHRS3.lat', 'AHRS3.lng', 'GLOBAL_POSITION_INT.lat',
'GLOBAL_POSITION_INT.lon', 'TERRAIN_REPORT.lat','TERRAIN_REPORT.lon', 'GPS_RAW_INT.lat',
'LOCAL_POSITION_NED.x','MISSION_ITEM_INT.x','POSITION_TARGET_GLOBAL_INT.lat_int','TERRAIN_REPORT.lat']

In [45]:
# 'AHRS3.lat', 'AHRS3.lng'
# create new dataframe with only ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon'] 
dfGPS = dfDrop[['AHRS3.lat', 'AHRS3.lng']]
dfDropGPS = dfGPS.dropna()
#dfDropGPS.head()
# 'VFR_HUD.groundspeed','VFR_HUD.heading'
dfDropGPS['AHRS3.lat'] = dfDropGPS['AHRS3.lat'].apply(addDec)
dfDropGPS['AHRS3.lng'] = dfDropGPS['AHRS3.lng'].apply(addDec)
dfDropGPS.head()
# 
dfDropGPS.to_csv('data//GPS-AHRS3.csv')

In [55]:
# 'GLOBAL_POSITION_INT.lat', 'GLOBAL_POSITION_INT.lon'
# create new dataframe with only ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon'] 
dfGPS = dfDrop[['GLOBAL_POSITION_INT.lat', 'GLOBAL_POSITION_INT.lon']]
dfDropGPS = dfGPS.dropna()
dfDropGPS.head(50)
# 'VFR_HUD.groundspeed','VFR_HUD.heading'
# dfDropGPS['GLOBAL_POSITION_INT.lat'] = dfDropGPS['GLOBAL_POSITION_INT.lat'].apply(addDec)
# dfDropGPS['GLOBAL_POSITION_INT.lon'] = dfDropGPS['GLOBAL_POSITION_INT.lon'].apply(addDec)
# dfDropGPS.head()
# # 
# dfDropGPS.to_csv('data//GPS-GLOBAL_POSITION_INT.csv')

,GLOBAL_POSITION_INT.lat,GLOBAL_POSITION_INT.lon
0,0.004025,895
1,0.004025,895
2,0.004025,895
3,0.004025,895
4,0.004025,895
5,0.004025,895
6,0.004025,895
7,0.004025,895
8,0.004025,895
9,0.004025,895


## Additional Workflows

In [ ]:
for c in df.columns:
    print(c)

In [5]:
# drop NaN row values
df = df.dropna()
df.shape

(2265, 487)

In [ ]:
coordFields = ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon', 'AHRS3.lat', 'AHRS3.lng', 'GLOBAL_POSITION_INT.lat',
               'GLOBAL_POSITION_INT.lon', 'TERRAIN_REPORT.lat','TERRAIN_REPORT.lon' ]
df['Unnamed: 237']

In [40]:
coordFields = ['ADSB_VEHICLE.lat', 'ADSB_VEHICLE.lon', 'AHRS3.lat', 'AHRS3.lng', 'GLOBAL_POSITION_INT.lat',
'GLOBAL_POSITION_INT.lon', 'TERRAIN_REPORT.lat','TERRAIN_REPORT.lon', 'GPS_RAW_INT.lat',
'LOCAL_POSITION_NED.x','MISSION_ITEM_INT.x','POSITION_TARGET_GLOBAL_INT.lat_int','TERRAIN_REPORT.lat']
for i, c in enumerate(coordFields):
    print(f'{c} - {df[c]}')


ADSB_VEHICLE.lat - 0        41.95869
1        41.95869
2        41.95869
3        41.95869
4        41.95869
           ...   
19311    43.00300
19312    43.00300
19313    43.00300
19314    43.00300
19315    43.00300
Name: ADSB_VEHICLE.lat, Length: 19316, dtype: float64
ADSB_VEHICLE.lon - 0       -83.307133
1       -83.307133
2       -83.307133
3       -83.307133
4       -83.307133
           ...    
19311   -82.295552
19312   -82.295552
19313   -82.295552
19314   -82.295552
19315   -82.295552
Name: ADSB_VEHICLE.lon, Length: 19316, dtype: float64
AHRS3.lat - 0        42.54606
1        42.54606
2        42.54606
3        42.54606
4        42.54606
           ...   
19311    42.53768
19312    42.53768
19313    42.53768
19314    42.53768
19315    42.53768
Name: AHRS3.lat, Length: 19316, dtype: float64
AHRS3.lng - 0       -83.319518
1       -83.319518
2       -83.319518
3       -83.319518
4       -83.319518
           ...    
19311   -83.318531
19312   -83.318531
19313   -83.318531
19314  

In [41]:
dfDrop['GLOBAL_POSITION_INT.lat']= dfDrop['GLOBAL_POSITION_INT.lat'].astype(str)

In [ ]:
coordFields = ['GLOBAL_POSITION_INT.lat', 'GLOBAL_POSITION_INT.lon']
# Coord fields not contain valid geometry 
# 'GLOBAL_POSITION_INT.lat', 'GLOBAL_POSITION_INT.lon', 'TERRAIN_REPORT.lat','TERRAIN_REPORT.lon' ]

dfDrop['GLOBAL_POSITION_INT.lat'] = dfDrop['GLOBAL_POSITION_INT.lat'].apply(addDec)

#dfFormat = fixCoords(dfDrop, coordFields)
dfFormat.head()

In [125]:
dfFormat.to_csv('data//06-17_08-58-07_edit.csv')